In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry 
import ipywidgets as widgets
from ipywidgets import interact, interactive
import IPython
from IPython.display import display, Markdown
import assignment_2_2023.msg
from assignment_2_2023.msg import PlanningAction
from assignment_2_2023.msg import Info
import actionlib
import actionlib.msg



In [2]:
def on_feedback(action_feedback):
    global send_coordinates
    
    # We can preempt the robot only if the state machine is in the "ACTIVE" status
    if(action_feedback.status.status == action_feedback.status.ACTIVE):
        send_coordinates.description = "Cancel"
        send_coordinates.style.button_color = 'red'
        
    else:
        send_coordinates.description = "Send Coordinates"
        send_coordinates.style.button_color = 'green'
        


In [3]:

def on_result(action_result):
    global send_coordinates
    # We can preempt the robot only if the state machine is not in the "SUCCEEDED" or in the "PREEMPTED" status
    if(not(action_result.status.status == action_result.status.SUCCEEDED or action_result.status.status == action_result.status.PREEMPTED)):
        send_coordinates.description = "Cancel"
        send_coordinates.style.button_color = 'red'
        
    else:
        send_coordinates.description = "Send Coordinates"
        send_coordinates.style.button_color = 'green'



In [4]:
def on_odom(msg):
    global info, pub_info, hbox_1, hbox_2, info_old
    pos = msg.pose.pose.position
	
	# Get the current velocity
    vel_linear_x = msg.twist.twist.linear.x
    vel_angular_z = msg.twist.twist.angular.z
    
    # Prepare and publish the info message
    info = Info()
    info.x = pos.x
    info.y = pos.y
    info.vel_linear_x = vel_linear_x
    info.vel_angular_z = vel_angular_z
    pub_info.publish(info)
    if(abs(info_old.x-info.x) >= 0.1 or abs(info_old.y-info.y) >= 0.1):
        hbox_1.children = ([widgets.Label("{:.1f}".format(info.x)),])
        hbox_2.children = ([widgets.Label("{:.1f}".format(info.y)),])
        info_old = info

In [5]:
global client, pub_info, info_old, send_coordinates
info_old = Info()
info_old.x = 0.0
info_old.y = 0.0
hbox_1 = widgets.HBox()
hbox_2 = widgets.HBox()
send_coordinates = widgets.Button()
rospy.init_node('client_node')

# sub_odom is a subscriber that receives Odometry data
sub_odom = rospy.Subscriber("/odom", Odometry, on_odom)

# Subscriber for feedback/status
sub_feedback = rospy.Subscriber("/reaching_goal/feedback",assignment_2_2023.msg.PlanningActionFeedback, on_feedback)
    
# Subscriber for result
sub_result = rospy.Subscriber("/reaching_goal/result",assignment_2_2023.msg.PlanningActionResult, on_result)

# info_pub is a publisher that sends the info (x,y,vel_linear_x, vel_angular_z)
pub_info = rospy.Publisher('/info_pos_vel', Info, queue_size = 1)

client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()



True

In [6]:

def on_coordinates_sent(b):
    global x_coordinate, y_coordinate, client, info, vbox, html_code_vertical_shorter
    if(b.description == "Cancel"):
        b.description = "Send Coordinates"
        b.style.button_color = 'green'
        client.cancel_goal()
        
        hbox = widgets.HBox()
        hbox_try0 = widgets.HBox((widgets.Label(value = "Deleted"),))
        hbox_try1 = widgets.HBox((widgets.Label("{:.1f}".format(info.x)),))
        hbox_try2 = widgets.HBox((widgets.Label("{:.1f}".format(info.y)),))
        hbox_try0.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try1.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try2.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')


        hbox.children = ([widgets.HTML(html_code_vertical_shorter), hbox_try0, widgets.HTML(html_code_vertical_shorter), hbox_try1, widgets.HTML(html_code_vertical_shorter), hbox_try2, widgets.HTML(html_code_vertical_shorter)])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
    elif(b.description == "Send Coordinates"):
        b.description = "Cancel"
        b.style.button_color = 'red'
        goal = assignment_2_2023.msg.PlanningGoal()
        goal.target_pose.pose.position.x = x_coordinate.value
        goal.target_pose.pose.position.y = y_coordinate.value
        
        hbox = widgets.HBox()
        hbox_try0 = widgets.HBox((widgets.Label("Start"),))
        hbox_try1 = widgets.HBox((widgets.Label(str(x_coordinate.value)),))
        hbox_try2 = widgets.HBox((widgets.Label(str(y_coordinate.value)),))
        hbox_try0.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try1.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
        hbox_try2.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')

        hbox.children = ([widgets.HTML(html_code_vertical_shorter), hbox_try0, widgets.HTML(html_code_vertical_shorter), hbox_try1, widgets.HTML(html_code_vertical_shorter), hbox_try2, widgets.HTML(html_code_vertical_shorter)])
        vbox.children = vbox.children + tuple([hbox,])
        vbox
        
        client.send_goal(goal)
        

In [7]:
global x_coordinate, y_coordinate, send_coordinates, vbox, hbox_1, hbox_2, html_code_vertical_shorter, info

# Important html lines used later to create the layout of the tables
html_code_vertical = '<div style="border-left: 1px solid #000; height: 100px;"></div>'
html_code_vertical_shorter = '<div style="border-left: 1px solid #000; height: 20px;"></div>'
html_code_horizontal = '<hr style="border-top: 1px solid #000;">'

# Float sliders declarations
x_coordinate = widgets.FloatSlider( 
value = "{:.1f}".format(info.x),
min = -9.0, 
max = 9.0, 
step = 0.1,
description = 'X:', 
disabled = False, 
continuous_update = False, 
orientation = 'horizontal', 
readout = True
)
y_coordinate = widgets.FloatSlider( 
value = "{:.1f}".format(info.y),
min = -9.0, 
max = 9.0, 
step = 0.1,
description = 'Y:', 
disabled = False, 
continuous_update = False, 
orientation = 'horizontal', 
readout = True
)

# Markdown with HTML syntax for font and size
introduction = f'<div style="font-family: Arial; font-size: 18px;">Choose X and Y coordinates to be reached from the robot</div>'
# This part is where the user can choose the coordinates
display(Markdown(introduction))
"""
The user can choose here the coorddinates for the robot to reach, thanks to two float sliders.
To send these coordinates to the robot, the user can click on the button "Send Coordinates".
When the robot is moving towards the coordinates, the button will change to "Cancel", and the user
has the possibility to cancel the operation by clicking on the button; in that case the button will
change back to "Send Coordinates".
"""
vbox_floatsliders = widgets.VBox()
vbox_floatsliders.layout = widgets.Layout(width='100%', height='100%', overflow_y='hidden', justify_content='center', align_items='center')
send_coordinates = widgets.Button(description="Send Coordinates", layout=widgets.Layout(width='50%', height='60px', color='black', font_weight='bold'))
send_coordinates.style.button_color = 'green'
send_coordinates.style.font_weight = 'bold'
hbox_button = widgets.HBox()
hbox_button.layout = widgets.Layout(width='75%', height='100%', overflow_y='hidden', justify_content='center', align_items='center')
hbox_button.children = [send_coordinates]
vbox_floatsliders.children = (x_coordinate, y_coordinate, hbox_button)
display(vbox_floatsliders)
send_coordinates.on_click(on_coordinates_sent)

current_coordinates_text = f'<div style="font-family: Arial; font-size: 18px;">Current coordinates of the robot</div>'
# This part shows the current coordinates of the robot
display(Markdown(current_coordinates_text))
"""
Here it is possible to look at the current position of the robot, with the x and y coordinates shown in the table below.
The coordinates are updated in real time, rounded to the first decimal.
"""
hbox_0 = widgets.HBox((widgets.Label("Coordinates:"),))
hbox_0.layout = widgets.Layout(width='33%', height='60px', overflow_y='hidden',justify_content='center', align_items='center')
hbox_1 = widgets.HBox((widgets.Label("{:.1f}".format(info.x)),))
hbox_1.layout = widgets.Layout(width='33%', height='60px', overflow_y='hidden', justify_content='center', align_items='center')
hbox_2 = widgets.HBox((widgets.Label("{:.1f}".format(info.y)),))
hbox_2.layout = widgets.Layout(width='33%', height='60px', overflow_y='hidden', justify_content='center', align_items='center')

hbox_coordinates = widgets.HBox()
hbox_coordinates.layout = widgets.Layout(width='100%', height='60px', overflow_y='hidden', )
vbox_coordinates = widgets.VBox()
vbox_coordinates.layout = widgets.Layout(width='100%', height='100px', overflow_y='hidden')
hbox_coordinates.children = [widgets.HTML(html_code_vertical), hbox_0, widgets.HTML(html_code_vertical), hbox_1, widgets.HTML(html_code_vertical), hbox_2, widgets.HTML(html_code_vertical)]
vbox_coordinates.children = [widgets.HTML(html_code_horizontal), hbox_coordinates, widgets.HTML(html_code_horizontal)]

display(vbox_coordinates)

sets_of_coordinates_text = f'<div style="font-family: Arial; font-size: 18px;">Here the sets of sent and canceled goals are shown</div>'
# Markdown for the sets of goals reached/cancelled
display(Markdown(sets_of_coordinates_text))
"""
This is done for the coordinates reached by the robot
A table is created with the coordinates reached by the robot, created with vboxes and hboxes
"""
vbox = widgets.VBox()
hbox = widgets.HBox()
hbox_try0 = widgets.HBox((widgets.Label("Operation"),))
hbox_try1 = widgets.HBox((widgets.Label("X"),))
hbox_try2 = widgets.HBox((widgets.Label("Y"),))
hbox_try0.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
hbox_try1.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')
hbox_try2.layout = widgets.Layout(width='33%', height='20px', overflow_y='hidden', justify_content='center', align_items='center')

hbox.children = ([widgets.HTML(html_code_vertical_shorter), hbox_try0, widgets.HTML(html_code_vertical_shorter), hbox_try1, widgets.HTML(html_code_vertical_shorter), hbox_try2, widgets.HTML(html_code_vertical_shorter)])
vbox.children = tuple([widgets.HTML(html_code_horizontal), hbox, widgets.HTML(html_code_horizontal),])
display(vbox)



<div style="font-family: Arial; font-size: 18px;">Choose X and Y coordinates to be reached from the robot</div>

<div style="font-family: Arial; font-size: 18px;">Current coordinates of the robot</div>

<div style="font-family: Arial; font-size: 18px;">Here the sets of sent and canceled goals are shown</div>